In [ ]:
import sys
sys.path.insert(0, '.')

import numpy as np
import keras
from umap import UMAP
import hdbscan
import matplotlib.pyplot as plt

# Import custom layers from VAE.py (required for loading the encoder)
from VAE import ClipLayer, Sampling

# ========== LOAD DATA ==========
# Load k-mer frequency data (2,762 features per sequence)
# Features: length + 6-mers(2080) + 5-mers(512) + 4-mers(136) + 3-mers(32) + GC(1)
data = np.load('./Data/all_multimer_frequencies_l5000_shuffled.npy')[:100_000]

# Skip the length column (column 0), keep only k-mer frequencies and GC
# This matches what VAE.py expects: 2,761 features
X_freq = data[:, 1:].astype(np.float32)

print(f'Loaded {X_freq.shape[0]} sequences with {X_freq.shape[1]} features')

# ========== LOAD VAE ENCODER ==========
encoder = keras.saving.load_model(
    'vae_encoder_final.keras',
    custom_objects = {'ClipLayer': ClipLayer, 'Sampling': Sampling}
)
print(f'Loaded encoder with output shape: {encoder.output_shape}')

2025-11-28 17:32:29.224492: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-28 17:32:29.259918: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/torben/Clustering/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: './all_multimer_frequencies_l5000_shuffled.npy'

In [ ]:
# ========== ENCODE DATA WITH VAE ==========
# Get the latent mean (z_mean) from the encoder
# Encoder outputs: [z_mean, z_log_var, z]
z_mean, z_log_var, z = encoder.predict(X_freq, batch_size = 8192, verbose = 1)

# Use z_mean for clustering (deterministic representation)
latent = z_mean
print(f'Latent embedding shape: {latent.shape}')

In [ ]:
# ========== HDBSCAN CLUSTERING ==========
# Cluster directly on VAE latent space (256 dimensions)
clusterer = hdbscan.HDBSCAN(
    min_cluster_size = 50,
    min_samples = 5,
    metric = 'euclidean'
)
labels = clusterer.fit_predict(latent)

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_noise = (labels == -1).sum()

print(f'Found {n_clusters} clusters')
print(f'Noise points: {n_noise} ({100 * n_noise / len(labels):.1f}%)')

In [ ]:
# ========== 2D UMAP FOR VISUALIZATION ==========
# Reduce latent space to 2D for plotting
reducer_2d = UMAP(
    n_components = 2,
    metric = 'euclidean',
    n_neighbors = 15,
    min_dist = 0.1,
    verbose = True,
    random_state = 42
)
embedding_2d = reducer_2d.fit_transform(latent)
print(f'2D embedding shape: {embedding_2d.shape}')

In [ ]:
# ========== VISUALIZATION ==========
fig, ax = plt.subplots(figsize = (12, 10))

# Plot noise points in gray
noise_mask = labels == -1
ax.scatter(
    embedding_2d[noise_mask, 0],
    embedding_2d[noise_mask, 1],
    c = 'lightgray',
    s = 1,
    alpha = 0.3,
    label = 'Noise'
)

# Plot clustered points
scatter = ax.scatter(
    embedding_2d[~noise_mask, 0],
    embedding_2d[~noise_mask, 1],
    c = labels[~noise_mask],
    cmap = 'tab20',
    s = 1,
    alpha = 0.5
)

ax.set_xlabel('UMAP 1')
ax.set_ylabel('UMAP 2')
ax.set_title(f'VAE Latent Space Clustering ({n_clusters} clusters, {100 * n_noise / len(labels):.1f}% noise)')
plt.colorbar(scatter, ax = ax, label = 'Cluster')
plt.tight_layout()
plt.show()

In [ ]:
# ========== PAIRWISE DISTANCES IN LATENT SPACE ==========
from scipy.spatial.distance import pdist

# Take first 1,000 samples and compute pairwise distances
z_sample = z[:1000]
distances = pdist(z_sample, metric = 'euclidean')

print(f'Number of pairwise distances: {len(distances):,}')
print(f'Distance range: {distances.min():.2f} - {distances.max():.2f}')
print(f'Mean distance: {distances.mean():.2f}, Std: {distances.std():.2f}')

# Plot histogram
fig, ax = plt.subplots(figsize = (10, 6))
ax.hist(distances, bins = 100, edgecolor = 'none', alpha = 0.7)
ax.set_xlabel('Euclidean Distance')
ax.set_ylabel('Count')
ax.set_title('Pairwise Distances in VAE Latent Space (z, n=1000)')
ax.axvline(distances.mean(), color = 'red', linestyle = '--', label = f'Mean: {distances.mean():.2f}')
plt.legend()
plt.tight_layout()
plt.show()